In [ ]:
import os
import pathlib
import site

In [ ]:
import cv2
import numpy as np

In [ ]:
from pytorch_lightning import Trainer, seed_everything

In [ ]:
from anomalib.config import get_configurable_parameters
from anomalib.data import get_datamodule
from anomalib.models import get_model
from anomalib.utils.callbacks import LoadModelCallback, get_callbacks

In [ ]:
from evaluator import Evaluator

## Config

In [ ]:
"""
SET MODEL NAME
"""
model_name = "padim"
config_path = f"./anomalib/models/{model_name}/config.yaml"
config = get_configurable_parameters(model_name=model_name, config_path=config_path)

In [ ]:
"""
DATASET SETTING 
"""
config.dataset.name = "mvtec"
config.dataset.path = "./datasets/MVTec/"
config.dataset.category = "bottle"

config.dataset.train_batch_size = 8
config.dataset.eval_batch_size = 8
config.dataset.num_workers = 0

# config.dataset.image_size =  [384, 384]

dict(config.dataset)

In [ ]:
dict(config.logging)

In [ ]:
config.metrics.pixel = []

dict(config.metrics)

In [ ]:
"""
MODEL SETTING

REF: https://github.com/JohnnyHopp/PaDiM-EfficientNetV2/blob/master/main.py
"""
config.model.backbone = "resnet18"
config.model.layers = ['layer1.-1', 'layer2.-1', 'layer3.-1']
config.model.n_features = 100

# config.model.backbone = "wide_resnet50_2"
# config.model.layers = ['layer1.-1', 'layer2.-1', 'layer3.-1']
# config.model.n_features = 550

# config.model.backbone = "tf_efficientnet_b5_ns"
# config.model.layers = ['blocks.1.-1', 'blocks.3.-1', 'blocks.4.-1']
# config.model.n_features = 100

# config.model.backbone = "tf_efficientnet_b7_ns"
# config.model.layers = ['blocks.1.-1', 'blocks.3.-1', 'blocks.4.-1']
# config.model.n_features = 100

# config.model.backbone = "tf_efficientnetv2_m_in21k"
# config.model.layers = ['blocks.2.-1', 'blocks.3.-1', 'blocks.4.-1']
# config.model.n_features = 400

# config.model.backbone = "convnext_base_in22ft1k"
# config.model.layers = ['stages_0.blocks.-1', 'stages_1.blocks.-1', 'stages_2.blocks.-1']
# config.model.n_features = 600
# config.model.input_size =  [384, 384]

dict(config.model)

In [ ]:
dict(config.optimization)

In [ ]:
config.project.path = f"./results/{config.dataset.category}_{model_name}_{config.model.backbone}_n{config.model.n_features}"

dict(config.project)

In [ ]:
config.trainer.default_root_dir = f"./results/{config.dataset.category}_{model_name}_{config.model.backbone}_n{config.model.n_features}"

dict(config.trainer)

In [ ]:
dict(config.visualization)

## Generate Fake Mask

In [ ]:
# test_dir_path = f"{config.dataset.path}{config.dataset.category}/test/"
# ground_truth_dir_path = f"{config.dataset.path}{config.dataset.category}/ground_truth/"
# bad_labels = [d for d in os.listdir(test_dir_path) if os.path.isdir(os.path.join(test_dir_path, d)) and d != 'good']

# for bad_label in bad_labels:
#     current_test_dir_path = f"{test_dir_path}/{bad_label}/"
#     current_ground_truth_dir_path = f"{ground_truth_dir_path}/{bad_label}/"
#     img_labels = [f for f in os.listdir(current_test_dir_path) if os.path.isfile(os.path.join(current_test_dir_path, f))]

#     if not os.path.exists(current_ground_truth_dir_path):
#         os.makedirs(current_ground_truth_dir_path)
        
#     for img_label in img_labels:
#         test_img_path = f"{current_test_dir_path}{img_label}"
#         test_img = cv2.imread(test_img_path, cv2.IMREAD_GRAYSCALE)
        
#         black_img = np.zeros(test_img.shape, dtype=np.uint8)
        
#         black_img_path = f"{current_ground_truth_dir_path}{img_label}"
#         cv2.imwrite(black_img_path, black_img)

## Train

In [ ]:
datamodule = get_datamodule(config)
model = get_model(config)
callbacks = get_callbacks(config)

In [ ]:
trainer = Trainer(**config.trainer, callbacks=callbacks)
trainer.fit(model=model, datamodule=datamodule)

# Test

In [ ]:
weight_file_path = trainer.checkpoint_callback.best_model_path
weight_file_path

In [ ]:
load_model_callback = LoadModelCallback(weights_path=weight_file_path)
trainer.callbacks.insert(0, load_model_callback)

res=trainer.test(model=model, datamodule=datamodule)

## Check

In [ ]:
# model.model

In [ ]:
preds_holder = callbacks[4].get_preds_holder()
evaluator = Evaluator.constrant_using_preds_holder(preds_holder)

display(f"max: {preds_holder.max}", f"min: {preds_holder.min}", f"threshold: {preds_holder.threshold}")
preds_holder.dataframe.head()

In [ ]:
evaluator.show_hist()

In [ ]:
evaluator.show_pdf()

In [ ]:
evaluator.show_hist_and_pdf()

In [ ]:
for layer, dim, size in zip(model.model.layers, model.model.feature_extractor.out_dims, model.model.feature_extractor.out_sizes):
    print(f"{layer} [dim:{dim}, size:{size}]")

## Appendix

In [ ]:
# import timm

In [ ]:
# timm.create_model(
#     "efficientnet_b5",
#     pretrained=True,
#     features_only=True,
#     exportable=True,
# )

In [ ]:
# timm.list_models(pretrained=True)